# readPTU_FLIM library
### **`What is available !`**
**`The library provides the capability to handle all PicoQuant's TCSPC Harps in T2 as well as T3 mode.`**
- PicoQuant uses a bespoke file format called PTU to store data from time-tag-time-resolved (TTTR) measurements.<br/>
- Current file format (.ptu) and is subsequent to the former .pt2 or .pt3 and can handle both T2 and T3 acquisition modes for a variety of TCSPC devices (MultiHarp, HydraHarp, TimeHarp, PicoHarp, etc.) <br/>
- At the moment, the library was tested for FLIM data obtained using MultiHarp, HydraHarp and PicoHarp. <br/>

### **`What is not available !`**
- Option to save data after processing. <br/>
#### !!! Help is only an email away for MATLAB implementation of the same library.


## Demo: Jupyter Notebook to work with PicoQuant PTU files

### Dependencies

* conda install numpy

* conda install matplotlib

* conda install numba

* readPTU_FLIM.py

In [ ]:
from readPTU_FLIM import PTUreader
import numpy as np
from matplotlib import pyplot as plt

## Select a PTU file to read
- As an example, first we import the readPTU_FLIM library. Then we should open the PTU file using a PTUreader() object. By constructing a PTUreader() object, automatically opens the file, reads the PTU file header and TTTR data in raw format. PTU file header contains important measurement information (PQ TCSPC unit, laser/pulse sync rate, FLIM record type, scanner type, etc.)<br/><br/> 
- **`Test_FLIM_image_daisyPollen_PicoHarp_2.ptu`** [Download test file here]([Download the test file here](https://drive.google.com/file/d/1bbtw0rZQk8HHlp8SYQlvMpdzuEEFIAZL/edit)<br> 
FLIM data was acquired using a PicoHarp (T3 mode) <br>
Exctiation Laser light: 485 nm<br>
Number of Detection Channels: 2<br/><br/>
- At the moment, PTUreader library contains implementation for reading imaging data from laser beam scanner (LSM) units.<br/><br/> 
- <u>**Readability for PTU/TTTR data acquired using Piezo Scanner not implemented YET!**<u> <br/><br/>


In [ ]:
# Choose a filename and create a
ptu_file  = PTUreader('Test_FLIM_image_daisyPollen_PicoHarp_2.ptu', print_header_data = False)

## How to access raw data from ptu_file object?

In [ ]:
# # Raw data can be accessed using ptu_file object
# sync    = ptu_file.sync    # Macro photon arrival time
# tcspc   = ptu_file.tcspc   # Micro photon arrival time (tcspc time bin resoultion)
# channel = ptu_file.channel # Detection channel of tcspc unit (<=8 for PQ hardware in 2019)
# special = ptu_file.special # Special event markers, for e.g. Frame, LineStart, LineStop, etc.

## Get flim_data_stack and intensity_image from raw TTTR data
 **`Note`**: once NEXT CODE block is executed raw TTTR data variables (sync, tcspc, channel, special) are deleted
- `flim_data_stack`: (pixX, pixY, spectral_detection_channel, tcspc_bins)
- `intensity_image`: (pixX, pixY)

**`For example`**:
How to get all the tcspc histogram data from all pixels from spectral detection channel 1?  (hint below!) <br /> 

```python
channel_1_data = flim_data_stack[:,:,0,:]
```

In [ ]:
#Get FLIM data stack
flim_data_stack, intensity_image = ptu_file.get_flim_data_stack()

## Plot Intensity image

In [ ]:
# %matplotlib inline
# #plot intensity image
# plt.imshow(intensity_image)
# plt.colorbar()

## Plot and Play ▶
- Please see Jupyter notebook for implementation
- Once the FLIM data stack is loaded, one could create custom gui for performing all sort of FLIM analysis routines as needed.
- As an example a simple intensity image is plotted 
- User can investigate the fluorescence decays after drawing a rectangle on the intensity image

In [ ]:
# Code for GUI
%matplotlib notebook

class RectangularROI:
    
    def __init__(self, decay_data, image_AX,decay_AX,decay_fig, tau_resolution):
        self.decay_data     = decay_data
        self.image_ax       = image_AX
        self.decay_ax       = decay_AX
        self.decay_fig      = decay_fig
        self.tau_resolution = tau_resolution
        self.image_ax.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.image_ax.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.xs  = None
        self.ys  = None
    def on_press(self, event):
        
        self.x0 = event.xdata
        self.y0 = event.ydata

    def on_release(self, event):

        self.x1 = event.xdata
        self.y1 = event.ydata
        self.x_indices = np.int_(np.ceil(np.abs(np.sort(np.array([self.x1,self.x0]))))) # [x1, x2]
        self.y_indices = np.int_(np.ceil(np.abs(np.sort(np.array([self.y0,self.y1]))))) # [y1, y2]
        self.ys = np.sum(self.decay_data[self.y_indices[0]:self.y_indices[1],self.x_indices[0]:self.x_indices[1],:], axis=0)
        self.ys = np.sum(self.ys, axis = 0)
        self.xs = np.linspace(0,decay_data.shape[2],decay_data.shape[2], dtype = np.int)*self.tau_resolution
        self.decay_ax.set_data(self.xs,self.ys)
        self.decay_fig.set_ylim(ymin = 0, ymax = np.max(self.ys)*10)
        self.decay_ax.fig.canvas.draw()


decay_data  = np.sum(flim_data_stack,axis = 2)        
tau_resolution = ptu_file.head["MeasDesc_Resolution"]*1e9

fig = plt.figure(figsize=(9,4))

image_ax = fig.add_subplot(121)
image_AX = image_ax.imshow(intensity_image,cmap="viridis") 
fig.colorbar(image_AX)
image_ax.set_aspect('auto')
plt.title('Draw a Rectangle on intensity image')

# Plot decay here
# default decay data Pixel (0,0)
plot_decay_data = decay_data[0,0,:]
tau = np.linspace(0,decay_data.shape[2],decay_data.shape[2], dtype = np.int)*(ptu_file.head["MeasDesc_Resolution"]*1e9)
decay_fig = fig.add_subplot(122)
decay_AX,  = decay_fig.plot(tau,plot_decay_data,'k-',label='Selected ROI Histogram', linewidth=1)
plt.yscale(value="log")
#plt.autoscale(enable=True, axis=1)
plt.axis([0, np.max(tau), 0, np.max(plot_decay_data)*10])
plt.xlabel('Time [ns]')
plt.ylabel('Intensity [counts]')
plt.title('TCSPC Decay')
plt.grid(True)
plt.legend()

plt.sca(decay_fig)
linebuilder = RectangularROI(decay_data,image_AX,decay_AX,decay_fig,tau_resolution)
plt.tight_layout()
plt.show()